# Tensorflow Keras Boston House Data

#### Import der Bibliotheken

In [ ]:
import numpy as np
import pandas as pd
import os
import imp

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.wrappers.scikit_learn import KerasRegressor
from tensorflow.keras.datasets import boston_housing

from sklearn.model_selection import cross_val_score
from sklearn.model_selection import KFold
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline


#### Laden der Daten

In [ ]:
header_list = ['CRIM','ZN','INDUS','CHAS','NOX','RM','AGE','DIS','RAD','TAX','PTRATIO','B','LSTAT','MEDV']
df_houses = pd.read_csv('https://archive.ics.uci.edu/ml/machine-learning-databases/housing/housing.data', names = header_list, sep='\s+')

#### Exploration der Daten

In [ ]:
df_houses.head(3)

####  Train und Test Data

In [ ]:
(train_data, train_targets), (test_data, test_targets) = boston_housing.load_data()

In [ ]:
print(train_data.shape)
print(test_data.shape)
print(train_targets[0:10])

#### Standardisierung der Daten

In [ ]:
mean = train_data.mean(axis=0)
train_data -= mean
std = train_data.std(axis=0)
train_data /= std
test_data -= mean
test_data /= std

#### Baseline NN-Modell

In [ ]:
def baseline_model():
    model = Sequential()
    model.add(Dense(64, activation='relu', input_shape=(train_data.shape[1],)))
    model.add(Dense(1))
    model.compile(optimizer='rmsprop', loss='mse', metrics=['mse','mae'])
    return model

#### Large NN-Modell

In [ ]:
def larger_model():
    model = Sequential()
    model.add(Dense(64, activation='relu', input_shape=(train_data.shape[1],)))
    model.add(Dense(64, activation='relu'))
    model.add(Dense(1))
    model.compile(optimizer='rmsprop', loss='mse', metrics=['mse','mae'])
    return model

#### Training

In [ ]:
num_epochs = 50

In [ ]:
network = baseline_model()
baseline_history = network.fit(train_data, train_targets, epochs=num_epochs, batch_size=1, verbose=1, validation_data=(test_data, test_targets))

In [ ]:
network = larger_model()
large_history = network.fit(train_data, train_targets, epochs=num_epochs, batch_size=1, verbose=1, validation_data=(test_data, test_targets))

In [ ]:
print(baseline_history.history['loss'][0:5])
print(large_history.history['loss'][0:5])

#### Small Model

In [ ]:
base_loss = baseline_history.history['loss']
base_val_loss = baseline_history.history['val_loss']
plot_history(base_loss, base_val_loss, 'training', 'test')

#### Large Model

In [ ]:
large_loss = large_history.history['loss']
large_val_loss = large_history.history['val_loss']
plot_history(large_loss, large_val_loss, 'training', 'test')

#### Utility Functions

In [ ]:
import numpy as np
from matplotlib.colors import ListedColormap
import matplotlib.pyplot as plt

def plot_decision_regions(X_train, X_test, y_train, y_test, classifier, xlabel, ylabel, resolution=0.02):
    
    print(type(X_test))
    
    if X_test is None:
        X = X_train
        y = y_train
        test_idx = None
    else:
        X = np.vstack((X_train, X_test))
        y = np.hstack((y_train, y_test))
        test_idx = range(y_train.shape[0], y_train.shape[0]+y_test.shape[0])

    # setup marker generator and color map
    markers = ('s', 'x', 'o', '^', 'v')
    colors = ('red', 'blue', 'lightgreen', 'gray', 'cyan')
    cmap = ListedColormap(colors[:len(np.unique(y))])

    # plot the decision surface
    x1_min, x1_max = X[:, 0].min() - 1, X[:, 0].max() + 1
    x2_min, x2_max = X[:, 1].min() - 1, X[:, 1].max() + 1
    xx1, xx2 = np.meshgrid(np.arange(x1_min, x1_max, resolution),
                           np.arange(x2_min, x2_max, resolution))
    Z = classifier.predict(np.array([xx1.ravel(), xx2.ravel()]).T)
    Z = Z.reshape(xx1.shape)
    plt.contourf(xx1, xx2, Z, alpha=0.3, cmap=cmap)
    plt.xlim(xx1.min(), xx1.max())
    plt.ylim(xx2.min(), xx2.max())

    for idx, cl in enumerate(np.unique(y)):
        plt.scatter(x=X[y == cl, 0], 
                    y=X[y == cl, 1],
                    alpha=0.8, 
                    c=colors[idx],
                    marker=markers[idx], 
                    label=cl, 
                    edgecolor='black')

    # highlight test samples
    if test_idx:
        # plot all samples
        X_test, y_test = X[test_idx, :], y[test_idx]

        plt.scatter(X_test[:, 0],
                    X_test[:, 1],
                    c='',
                    edgecolor='black',
                    alpha=1.0,
                    linewidth=1,
                    marker='o',
                    s=100, 
                    label='test set')

    plt.xlabel(xlabel)
    plt.ylabel(ylabel)
    plt.legend(loc='upper left')
    plt.tight_layout()

def plot_history(hist1, hist2, label1, label2):
    epochs = range(1, len(hist1))
    hist1 = hist1[1:]
    hist2 = hist2[1:]
    # b+ is for "blue cross"
    plt.plot(epochs, hist1, 'b+', label=label1)
    # "bo" is for "blue dot"
    plt.plot(epochs, hist2, 'bo', label=label2)
    plt.xlabel('Epochs')
    plt.ylabel('Loss')
    plt.legend()

    plt.show()